In [82]:
!pip3 install polygon

     ---------------------------------------- 0.0/111.3 kB ? eta -:--:--
     -------------------------------------- 111.3/111.3 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/75.9 kB ? eta -:--:--
     ---------------------------------------- 75.9/75.9 kB ? eta 0:00:00
     ---------------------------------------- 0.0/58.5 kB ? eta -:--:--
     ---------------------------------------- 58.5/58.5 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/85.6 kB ? eta -:--:--
     ---------------------------------------- 85.6/85.6 kB ? eta 0:00:00
     ---------------------------------------- 0.0/77.0 kB ? eta -:--:--
     ---------------------------------------- 77.0/77.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\ashkm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [102]:
from polygon import RESTClient
client = RESTClient(api_key="qAECEY64BgfArsBJID6VYCJ1k0asFEll", trace=False)
ticker = "AAPL"

In [96]:
import numpy as np
import datetime as dt

In [118]:
import pandas as pd

def get_stock_data_to_df(ticker):
    aggs = []
    B = client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2023-01-01", to="2023-06-13", limit=50000)
    # for a in B:
    #     aggs.append(a)
    # items = aggs
    data = []
    for item in B:
        # print(dir(item))
        data.append({
            "open": item.open,
            "high": item.high,
            "low": item.low,
            "close": item.close,
            "volume": item.volume,
            "vwap": item.vwap,
            "timestamp": item.timestamp,
            "transactions": item.transactions,
        
        })
    return pd.DataFrame(data)

In [144]:
def stationaryize(df : pd.DataFrame, time) -> pd.DataFrame:
    df['stationary'] = (np.log(df[time])).diff()
    df['timestamp_processed'] =  pd.to_datetime(df['timestamp'], unit='ms')
    df = df.dropna(how = 'any', axis = 0, )
    return df

def get_clean_data(ticker, open_or_close) -> pd.DataFrame:
    return stationaryize(get_stock_data_to_df(ticker.upper()), open_or_close)

In [145]:
aapl_df = get_clean_data("aapl", 'close')
# aapl_df = aapl_df.iloc[1:-1]
# aapl_df.head(-25)

In [146]:
aapl_df.head()

,open,high,low,close,volume,vwap,timestamp,transactions,stationary,timestamp_processed
1,130.87,131.17,130.87,131.10,8820.0,130.9552,1672736460000,157,0.000763,2023-01-03 09:01:00
2,131.18,131.24,131.17,131.17,2112.0,131.2088,1672736520000,53,0.000534,2023-01-03 09:02:00
3,131.19,131.29,131.19,131.28,3888.0,131.2208,1672736580000,90,0.000838,2023-01-03 09:03:00
4,131.28,131.46,131.28,131.46,5984.0,131.3279,1672736640000,88,0.001370,2023-01-03 09:04:00
5,131.36,131.44,131.30,131.30,9328.0,131.3368,1672736700000,157,-0.001218,2023-01-03 09:05:00


In [147]:
tsla_df = get_clean_data("aapl", 'close')
tsla_df.head()

,open,high,low,close,volume,vwap,timestamp,transactions,stationary,timestamp_processed
1,130.87,131.17,130.87,131.10,8820.0,130.9552,1672736460000,157,0.000763,2023-01-03 09:01:00
2,131.18,131.24,131.17,131.17,2112.0,131.2088,1672736520000,53,0.000534,2023-01-03 09:02:00
3,131.19,131.29,131.19,131.28,3888.0,131.2208,1672736580000,90,0.000838,2023-01-03 09:03:00
4,131.28,131.46,131.28,131.46,5984.0,131.3279,1672736640000,88,0.001370,2023-01-03 09:04:00
5,131.36,131.44,131.30,131.30,9328.0,131.3368,1672736700000,157,-0.001218,2023-01-03 09:05:00


In [148]:
nvda_df = get_clean_data("aapl", 'close')
nvda_df.head()

,open,high,low,close,volume,vwap,timestamp,transactions,stationary,timestamp_processed
1,130.87,131.17,130.87,131.10,8820.0,130.9552,1672736460000,157,0.000763,2023-01-03 09:01:00
2,131.18,131.24,131.17,131.17,2112.0,131.2088,1672736520000,53,0.000534,2023-01-03 09:02:00
3,131.19,131.29,131.19,131.28,3888.0,131.2208,1672736580000,90,0.000838,2023-01-03 09:03:00
4,131.28,131.46,131.28,131.46,5984.0,131.3279,1672736640000,88,0.001370,2023-01-03 09:04:00
5,131.36,131.44,131.30,131.30,9328.0,131.3368,1672736700000,157,-0.001218,2023-01-03 09:05:00


In [149]:
merged_try1_df = pd.merge(aapl_df[['timestamp_processed', 'stationary']], tsla_df[['timestamp_processed', 'stationary']], on='timestamp_processed', how='right').rename(columns = {'stationary_x':'apple', 'stationary_y':'tesla'})


In [174]:
def merge_df(ticker1, ticker2, open_or_close):
    df1 = get_clean_data(ticker1, open_or_close)
    df2 = get_clean_data(ticker2, open_or_close)
    return pd.merge(df1[['timestamp_processed', 'stationary']], df2[['timestamp_processed', 'stationary']], on='timestamp_processed', how='right').rename(columns = {'stationary_x': ticker1, 'stationary_y': ticker2}).dropna(how='any')




In [158]:
!pip3 install statsmodels

  Using cached statsmodels-0.14.1-cp310-cp310-win_amd64.whl (9.8 MB)



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\ashkm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [159]:
from statsmodels.tsa.stattools import grangercausalitytests

In [188]:
a= merge_df('goog','tsla', 'close')
a.head()

,timestamp_processed,goog,tsla
0,2023-01-03 09:01:00,0.004236,0.006977
3,2023-01-03 09:04:00,0.000667,-0.003598
4,2023-01-03 09:05:00,0.000000,-0.001426
5,2023-01-03 09:06:00,-0.000334,-0.001008
10,2023-01-03 09:11:00,0.000334,0.000169


In [186]:
test = grangercausalitytests(a[['goog', 'tsla']], 4)
print(test)



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=153.8478, p=0.0000  , df_denom=69718, df_num=1
ssr based chi2 test:   chi2=153.8544, p=0.0000  , df=1
likelihood ratio test: chi2=153.6849, p=0.0000  , df=1
parameter F test:         F=153.8478, p=0.0000  , df_denom=69718, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=88.7400 , p=0.0000  , df_denom=69715, df_num=2
ssr based chi2 test:   chi2=177.4928, p=0.0000  , df=2
likelihood ratio test: chi2=177.2673, p=0.0000  , df=2
parameter F test:         F=88.7400 , p=0.0000  , df_denom=69715, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=59.2552 , p=0.0000  , df_denom=69712, df_num=3
ssr based chi2 test:   chi2=177.7834, p=0.0000  , df=3
likelihood ratio test: chi2=177.5571, p=0.0000  , df=3
parameter F test:         F=59.2552 , p=0.0000  , df_denom=69712, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=46.2711 